In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_initial_data(years=None):
    url = "https://www.gt-world-challenge-europe.com/results?filter_season_id=0&filter_meeting_id=0&filter_race_id="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Извлекаем доступные сезоны
    seasons_options = soup.select('#filter_season_id option')
    seasons = {option.text.strip(): option['value'] for option in seasons_options if option['value']}

    # Фильтруем сезоны по указанным годам, если таковые имеются
    if years:
        seasons = {year: seasons[year] for year in years if year in seasons}
    
    # Извлекаем доступные встречи
    meetings_options = soup.select('#filter_meeting_id option')
    meetings = {option.text.strip(): option['value'] for option in meetings_options if option['value']}
    
    return seasons, meetings


def parse_years(input_years):
    if isinstance(input_years, list):
        return [str(year) for year in input_years]
    elif isinstance(input_years, str) and '-' in input_years:
        start_year, end_year = map(int, input_years.split('-'))
        return [str(year) for year in range(start_year, end_year - 1, -1)]
    elif input_years:
        return [str(input_years)]
    else:
        return None


def get_race_data(input_years=None):
    base_url = "https://www.gt-world-challenge-europe.com/results"
    years = parse_years(input_years)
    seasons, meetings = fetch_initial_data(years)
    results = []

    for season_name, season_id in seasons.items():
        for meeting_name, meeting_id in meetings.items():
            url = f"{base_url}?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id="
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            race_options = soup.select('#filter_race_id option')

            races = []
            for option in race_options:
                if option['value']: # Исключаем пустые значения
                    races.append({"race_id": option['value'], "name": option.text.strip()})
            
            # Добавляем информацию о сезоне, встрече и гонках в результат
            results.append({
                "season": season_name,
                "season_id": season_id,
                "meeting": meeting_name,
                "meeting_id": meeting_id,
                "races": races
            })

    return results


def fetch_race_results(filter_season, filter_meeting, filter_race):
    # Получаем данные о сезонах и встречах
    seasons, meetings = fetch_initial_data()
    
    # Ищем соответствующие ID для сезона и встречи
    season_id = None
    for season, id in seasons.items():
        if season == filter_season:
            season_id = id
            break
    
    if season_id is None:
        raise ValueError(f"Сезон '{filter_season}' не найден.")
    
    meeting_id = None
    for meeting, id in meetings.items():
        if meeting == filter_meeting:
            meeting_id = id
            break
    
    if meeting_id is None:
        raise ValueError(f"Встреча '{filter_meeting}' не найдена.")
    
    # Используем get_race_data для получения информации о гонках
    race_data = get_race_data()
    race_id = None
    for data in race_data:
        if data['season_id'] == season_id and data['meeting_id'] == meeting_id:
            for race in data['races']:
                if race['name'] == filter_race:
                    race_id = race['race_id']
                    break
            break
    
    if race_id is None:
        raise ValueError(f"Гонка '{filter_race}' не найдена.")
    
    # Формирование запроса с найденными ID и получение данных гонки
    final_url = f"https://www.gt-world-challenge-europe.com/results?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id={race_id}"
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Парсинг данных гонки
    rows = soup.select('.table__body tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        data.append([col.text.strip() for col in cols if col.text.strip() != ''])
    
    # Создание DataFrame
    if not data:
        return pd.DataFrame()  # Возвращаем пустой DataFrame, если данных нет
    df_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    df = pd.DataFrame(data, columns=df_columns)
    
    return df



In [19]:
result_df = fetch_race_results("2023", "Monza", "Main Race after 2.30 hours")
print(result_df)

   Pos Car #       Class  ...      Time          Laps           Gap
0    1    98     Pro Cup  ...  1:47.532            77          None
1    2    32     Pro Cup  ...  1:47.574            77         4.331
2    3   998     Pro Cup  ...  1:47.850            77        13.003
3    4    63     Pro Cup  ...  1:47.540            77        22.431
4    5    25     Pro Cup  ...  1:47.559            77        26.056
5    6    40     Pro Cup  ...  1:48.004            77        35.098
6    7    21    Gold Cup  ...  1:47.803            77        44.630
7    8    51     Pro Cup  ...  1:47.780            77        48.491
8    9    54     Pro Cup  ...  1:48.007            77        48.951
9   10     5    Gold Cup  ...  1:47.806            77        58.834
10  11    30    Gold Cup  ...  1:48.035            77        59.685
11  12    96     Pro Cup  ...  1:47.234            77      1:02.191
12  13    87     Pro Cup  ...  1:48.301            77      1:05.562
13  14    71     Pro Cup  ...  1:47.933         

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_race_data(filter_season_id, filter_meeting_id, filter_race_id):
    # Словари для хранения соответствий ID
    seasons = {
        2024: "24",
        2023: "23",
        2022: "22",
        # Добавьте остальные необходимые соответствия
    }
    meetings = {
        "Barcelona": "207",
        "Misano": "203",
        # Добавьте остальные необходимые соответствия
    }
    
    # Проверяем, есть ли указанные ID в словарях
    season_value = seasons.get(filter_season_id)
    meeting_value = meetings.get(filter_meeting_id)
    
    if season_value is None or meeting_value is None:
        raise ValueError("Указаны неверные значения для сезона или встречи.")
    
    # URL для первого запроса (для получения списка сессий)
    url = f'https://www.gt-world-challenge-europe.com/results?filter_season_id={season_value}&filter_meeting_id={meeting_value}&filter_race_id='
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Поиск нужного filter_race_id по тексту ссылки
    race_options = soup.select('#filter_race_id option')
    race_value = None
    for option in race_options:
        if option.text.strip() == filter_race_id:
            race_value = option['value']
            break
    
    if race_value is None:
        raise ValueError(f"Сессия {filter_race_id} не найдена.")
    
    # URL для второго запроса (для получения данных сессии)
    final_url = f'https://www.gt-world-challenge-europe.com/results?filter_season_id={season_value}&filter_meeting_id={meeting_value}&filter_race_id={race_value}'
    
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Парсинг данных сессии
    rows = soup.select('.table__body tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        data.append([col.text.strip() for col in cols])
    
    # Создание DataFrame
    df_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    df = pd.DataFrame(data, columns=df_columns)
    
    return df


data = get_race_data(2023, "Barcelona", "Main Race")

data.head()


,Pos,Car #,Class,Drivers,Team,Car,Time,Laps,Gap
0,1,51,Pro Cup,"Alessio Rovera, Robert Shwartzman, Nicklas Nie...",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:41.540,88,
1,2,71,Pro Cup,"Antonio Fuoco, Daniel Serra, Davide Rigon",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:41.665,88,0.422
2,3,96,Pro Cup,"Thomas Preining, Laurin Heinrich, Dennis Olsen",Rutronik Racing,Porsche 911 GT3 R (992),1:41.562,88,2.328
3,4,777,Pro Cup,"Maro Engel, Luca Stolz, Fabian Schiller",Mercedes-AMG Team AlManar,Mercedes-AMG GT3 EVO,1:41.467,88,2.770
4,5,88,Pro Cup,"Raffaele Marciello, Jules Gounon, Timur Bogusl...",AKKODIS ASP Team,Mercedes-AMG GT3 EVO,1:42.162,88,4.413
